In [4]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from konlpy.tag import Okt
from collections import Counter




In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/tripadviser_review.csv")




In [14]:
import re

def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

okt = Okt()  # 명사 형태소 추출 함수
nouns = okt.nouns(apply_regular_expression(df['text'][0]))
nouns
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

from sklearn.feature_extraction.text import CountVectorizer

def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    okt = Okt()  # 형태소 추출
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
    return nouns

vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(df['text'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

word_count_dict = dict(zip(word_list, count_list))
word_count_dict

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}

def rating_to_label(rating):
    if rating > 3:
        return 1
    else:
        return 0
    
df['y'] = df['rating'].apply(lambda x: rating_to_label(x))
from sklearn.model_selection import train_test_split

x = tf_idf_vect
y = df['y']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# fit in training set
lr = LogisticRegression(random_state = 0)
lr.fit(x_train, y_train)

# predict in test set
y_pred = lr.predict(x_test)

# confusion matrix

from sklearn.metrics import confusion_matrix

confu = confusion_matrix(y_true = y_test, y_pred = y_pred)

plt.figure(figsize=(4, 3))
sns.heatmap(confu, annot=True, annot_kws={'size':15}, cmap='OrRd', fmt='.10g')
plt.title('Confusion Matrix')
plt.show()


KeyError: 'rating'

In [ ]:
df = pd.read_csv("first_article.csv")

In [17]:
vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.transform(df['text'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

word_count_dict = dict(zip(word_list, count_list))
word_count_dict

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}

x_feeds = tf_idf_vect

# predict in test set
y_pred = lr.predict(x_feeds)

# confusion matrix


NotFittedError: Vocabulary not fitted or provided

In [16]:
df['text']

0                     자기는 고졸에 행시로 여기까지 왔으면서  사다리 걷어차기! 
1                   국회의원 기본급 절반삭감, 활동비 없애기, 연금 폐지부터 해라 
2     난 김동연 이사람이 참 합리적인 사람으로보여.근디 혼자서는 힘든데.그래도 윤씨나 최...
3     공무원 정년 폐지...ㅋ 아예 국회의원 임기 폐지 들고 나오지  젊은 세대 죽어 나...
4                      건들지 말아야 할 공직사회를 건드리시다니...용감합니다. 
5     김동연화이팅!!!! 이런 사람이 많이 나와서  대한민국을 바꾸어 놓아야 하는데  아...
6       ●공무원 철밥통은 깨야한다. ●공무원의 복지부동이 나라 밍친다. ●꼬옥~실천해주세요 
7             이분은 그래도 양심과 도덕적이지 윤석열 최재형 은 직을 이용해 출세하려고 
8     직급 계급 건너뛰는 제도 전부 없애라 사관학교  경찰대  7급 5급  외시 행시 없...
9     실망이다  국민들은  힘들고 어려운데 공무원이 너무많고 예산 낭비다.  국민을 생각...
10                             이런 공약이 제대로 이행되어야 나라가 산다 
11                                      뭐  애들 대통놀이 하니? 
12     폐지가 맞다.  행정고시도  사법고시와 똑 같다.  순기능 보다는 악기능이 더 많다. 
13                                    공무원 정년 없애면 어떡하니? 
14                                         집에가서 잠이나 자라 
15                                       화이팅. 청와대로 보내자 
16                  넌 안된다는거 알면서도 왜 나오는거냐? 스스로도 알텐데 말이야 
17    김동연 자신도 행정고시를 통해서... 5급 공무원으로 공직을 시작했는데...